# Assignment 7

### Part A

In [253]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [254]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [255]:
def expected_return(portfolio, rf, mkt_rp):
    # Calculate the expected return for each security
    portfolio['Expected Return'] = rf + portfolio['Beta'] * (mkt_rp)

    # Calculate the total investment to get weights
    total_investment = portfolio['Amount Invested'].sum()

    # Calculate the weighted return for each security
    portfolio['Weighted Return'] = (portfolio['Expected Return'] * portfolio['Amount Invested']) / total_investment

    # Calculate the expected return of the portfolio
    portfolio_return = portfolio['Weighted Return'].sum()

    return portfolio_return

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [256]:
def volatility(P, cor):
    # Calculate the weights of each stock in the portfolio
    total_investment = P['Amount Invested'].sum()
    P['Weight'] = P['Amount Invested'] / total_investment

    # Extract the volatilities and weights
    volatilities = P['Volatility'].values
    weights = P['Weight'].values

    # Create a correlation matrix
    cor_matrix = np.full((len(weights), len(weights)), cor)
    np.fill_diagonal(cor_matrix, 1)

    # Create a covariance matrix from the standard deviations and the correlation matrix
    cov_matrix = np.outer(volatilities, volatilities) * cor_matrix

    # Calculate portfolio variance
    portfolio_variance = np.dot(weights, np.dot(cov_matrix, weights))

    # Calculate portfolio volatility
    portfolio_volatility = np.sqrt(portfolio_variance)
    
    return portfolio_volatility

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [257]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    # Calculate the expected return of the market portfolio
    expected_return_market = rf + mkt_rp

    # Calculate the expected return of the portfolio using the CML formula
    expected_return_portfolio = rf + ((expected_return_market - rf) / mkt_vol) * vol

    return expected_return_portfolio


### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [258]:
def CAPM_beta(return_path, factors_path):
    # Load the returns DataFrame
    df1 = pd.read_csv(return_path)
    df2 = pd.read_csv(factors_path)
    
    df2['Mkt-RF'] = df2['Mkt-RF'] / 100
    df2['RF'] = df2['RF'] / 100
    
    # Split the 'date' column into 'year' and 'month'
    df2['year'] = df2['date'].astype(str).str[:4].astype(int)
    df2['month'] = df2['date'].astype(str).str[4:].astype(int)
    
    # Drop the original 'date' column
    df2.drop('date', axis=1, inplace=True)
    
    # Merge the returns DataFrame with the factors DataFrame on 'year' and 'month'
    df_merged = pd.merge(df1, df2, on=['year', 'month'], how='left')
    
    # Calculate excess returns for the stocks and the market
    df_merged['Excess_RET'] = df_merged['RET'] - df_merged['RF']
    
    # Group by CUSIP and calculate covariance and variance
    grouped = df_merged.groupby('CUSIP')
    market_var = df_merged['Mkt-RF'].var()

    # Calculate CAPM beta for each CUSIP
    beta = grouped['Excess_RET'].apply(lambda x: x.cov(df_merged['Mkt-RF']) / market_var)
    
    # Calculate expected return using the CAPM formula for each CUSIP
    exp_return = df2['RF'].mean() + beta * df2['Mkt-RF'].mean()

    # Combine the results into a DataFrame
    output = pd.DataFrame({
        "CUSIP": beta.index,
        "EXP_RETURN": exp_return,
        "BETA": beta.values
    })
    
    return output

In [259]:
CAPM_beta("monthlycrsp_mod7.csv", "F-F_Research_Data_Factors.csv")

CUSIP  EXP_RETURN      BETA
CUSIP                                   
00081T10  00081T10    0.020421  2.625012
00108M10  00108M10    0.007692  0.742881
00130H10  00130H10    0.011895  1.364285
00163U10  00163U10    0.009056  0.944552
00182C10  00182C10    0.011632  1.325491
...            ...         ...       ...
9959910    9959910    0.008050  0.795820
9972410    9972410    0.011101  1.246888
9972510    9972510    0.010595  1.172172
9984910    9984910    0.014720  1.781970
9985510    9985510    0.011976  1.376381

[3225 rows x 3 columns]

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [260]:
def slope(data):
    # Ensure that the data is sorted by BETA in ascending order
    data_sorted = data.sort_values(by='BETA')
    
    # Use np.polyfit to fit a line to the data
    beta = data_sorted['BETA']
    exp_return = data_sorted['EXP_RETURN']
    m, b = np.polyfit(beta, exp_return, 1)
    
    return m